In [10]:
import math

def getCos(m, u):
	compo = [m[i] * u[i] for i in range(len(m))]
	return sum(compo)

def normalize(profi):
	norm2 = sum([dim ** 2 for dim in profi])
	norm = math.sqrt(norm2)
	return [dim / norm2 for dim in profi] #norm(profi) == 1

def getProfi(row):
	idx = row[0]
	profi = [float(profiDim) for profiDim in row[1:]]
	nProfi = normalize(profi)
	return (idx, nProfi)

def getSim(MU):
	(mIdx, mProfi), (uIdx, uProfi) = MU
	sim = getCos(mProfi, uProfi)
	return (mIdx, uIdx, sim)

In [2]:
uProfi = spark.read.csv('/user/hz333/data/project/userMetaProfile.csv', header = False)
mProfi = spark.read.csv('/user/hz333/data/project/movieMetaProfile.csv', header = False)

In [11]:
#(mIdx, mProfi) => #(mIdx, [nMProfi])
movie = mProfi.rdd.map(getProfi)
#(uIdx, uProfi) => #(uIdx, [nMProfi])
user = uProfi.rdd.map(getProfi)
#(mIdx, [mProfi]), (uIdx, [uProfi]) => ((mIdx, [mProfi]), (uIdx, [uProfi]))
MU = movie.cartesian(user)
#((mIdx, [mProfi]), (uIdx, [uProfi])) => (mIdx, uIdx, sim)
simMat = MU.map(getSim)

In [13]:
simCSV = spark.createDataFrame(simMat, samplingRatio = 1)
simCSV.write.option('header', 'false').csv('/user/hz333/data/project/metaSim.csv')

In [12]:
simMat.filter(lambda x: (x[0] == '1' and x[1] == '1')).collect()

[('1', '1', 0.0003328362661453306)]